# In-depth explanation of the BERT model 

Transformer is a structure proposed by Google in 2017 for the encoding and decoding of machine translation. Later, it was widely used as an encoder because of its powerful coding ability and parallelism. Each Transformer module can be referred to as a layer in the model. At present, the commonly used model is composed of a stack of multiple transformers, and the semantic representation combined with the context is gradually extracted from low-level to high-level.
The input received by Transformer is a vector of word sequences of length nn $x = (x_1,…,x_n)$, $x \in R^{n \times d}x∈Rn×d$, where dd is the vector dimension. The output is the encoded vector $z = (z_1, …, z_n)$, $z \in R^{n \times d}z∈Rn×d$. Its structure is shown in the figure below. The transformer is composed of two sub-layers. The first sub-layer is a multi-head self-attention mechanism, and the second sub-layer is simple. Feed Forward Network (Feed Forward Network). In order to better train the model and accelerate convergence, residual connection and layer normalization will be performed after each sub-layer. The output of the previous layer is used as the input of the next layer, and the continuous stacking finally obtains the high-level semantic representation combined with the context.

![image.png](./../imgs/multi-head.png)

• Multi-Head Attention (MHA)

Multi-Head Attention is the core part of the transformer, which is derived from the attention mechanism (Attention). Attention first appeared in machine translation tasks [9], and then various variants were developed and widely used in various fields of NLP. Its method is to assign different weights to each word, and finally obtain new word and sentence representations through linear weighting. In fact, the sentence is implicitly structured, the important parts are extracted, noise is removed, and semantic information is better obtained. The MHA used in BERT is mainly based on Scaled Dot-Product Attention.
The input of Scaled Dot-Product Attention is query, key and value, which are recorded as $ Q \in R^{n \times d_{k}}Q∈Rn×dk; K \in R^{n \times d_{k }}K∈Rn×dk $ and $V \in R^{n \times d_{v}}V∈Rn×dv$, where QQ and KK have the same dimensions. The formula for calculating Attention weight is as follows:
$Attention(Q, K, V) = softmax(\frac{QK^{T}} {\sqrt(d_k)})V$ calculate the inner product and get a $n×n$ dimension matrix. The division of $d_k$ here is because the calculation of $QK^T$ involves the accumulation of products. For larger $d_k$, the result of the dot product is greatly increased, pushing the Softmax function to a region with minimal gradient (in order to clarify the point The reason why the product becomes larger, suppose that q and k are independent random variables with an average of 0 and a variance of 1, so their dot product is $ q\cdot k=\sum_{i=1}^{d_k}q_i\cdot k_i$, the mean is 0 and the variance is d_kdk). In order to counteract this influence, the paper uses $\frac{1}{\sqrt{d_k}}dk1$ to scale the dot product and eliminate the influence of dimensionality.
After that, use softmax to normalize each row of the matrix to get the corresponding weight, and perform a weighted sum of VV as the final output of the Attention module, as shown on the left of the following figure:

![image.png](./../imgs/multi-head-detail.png)

Multi-Head Attention (on the right of the figure above) is to perform multiple linear mappings on Q, K, and V, and combine the results of multiple Attentions as the final output. The purpose is to extract different features through different attention heads. , Just like there will be multiple convolution kernels in CNN.
For head_iheadi, the projection matrix corresponding to Q, K, and V is ${W_i}^Q \in R^{d_{model \times d_k}}$, ${W_i}^K \in R^{d_{model \times d_k}}$, ${W_i}^V \in R^{d_{model \times d_v}}WiQ∈Rdmodel×dk$, $WiK∈Rdmodel×dk, WiV∈Rdmodel×dv$, and project their corresponding Calculate Attention as input. Finally, the Attention output of the multi-head is spliced together, and the final output is projected into the space of dimension $d_{model}$ through full connection.

$head_i = Attention(Q{W_i}^Q, K{W_i}^K, V{W_i}^V), i = 1,…,h$

$MultiHead(Q,K,V)=Concat(head1,… ,headh)WO$

Let h be the number of multiple heads, $d_k = d_v = d_{model} / h$. As the number of heads increases, the dimensions of $d_k$ and $d_v$ will decrease proportionally, so the computational complexity of multi-head attention Similar to single head attention.

- Feed Forward Network (FFN)

The Feed Foward structure in Transformer is actually a two-layer full connection, plus the ReLU activation function [10]. The above multi-head attention is essentially linear weighting, so the main purpose of FFN is to add a non-linear module to the transformer to improve the fitting ability of the model.
Record the input as xx, then the output after FFN is:
$FFN(x) = max(0, xW_1+b_1)W_2 + b_2$

• Add & LayerNorm

Since the transformer is stacked in multiple layers in practical applications, and each layer of the transformer is composed of two sub-layers, in order to make the input distribution of each stage more stable and ensure the effective transmission of the gradient, an increase is added after the sub-layer The mechanism of residual connection [11] and Layernorm [12].
Remember that the input of sublayer is xx, and the final output passed to the next stage is:
$y = LayerNorm(x + Sublayer(x))$
Among them, LayerNorm is the normalization in the feature dimension. For more comparative research related to normalization, please refer to here.

• Difference with CNN and RNN

Convolutional Neural Networks (CNN) [13] and Recurrent Neural Networks (RNN) [14] and their variants LSTM, GRU, etc., are also commonly used text encoders in natural language processing. The recurrent neural network receives the word vectors $x_t$ at different moments one by one and combines the historical information $h_{t-1}$ to get the updated $h_t$. Its recursive structure is naturally suitable for processing sequential data such as text.

![image.png](./../imgs/rnn.png)

The convolutional neural network was initially used to process image data ($W \times H$ matrix data), because we represent the text sequence of length nn as a vector and it is also in the form of a matrix ($N \times dimension$). Therefore, you can use the kernel with the size of $k \times dimension$ to do convolution to obtain the information between adjacent words, similar to the traditional n-gram model. Finally, the final representation is obtained through max-over-time-pooling.

![image.png](./../imgs/textcnn.png)

Due to the recursive structure of RNN, it cannot achieve parallel computing, while both CNN and tranformer can increase the computing speed through parallelism. In addition, the long-term dependency problem (long-term dependency), that is, the model cannot capture the farther dependencies in the sequence, is a difficulty in natural language processing. We hope that the model can capture any position in the text through the shortest path. Dependence. From this dimension, the farthest path for RNN to obtain the dependency relationship at any position in the text sequence of length $n$ is $o(n)$, the farthest path of CNN is $o(log_k(n))$, and transfomer only Need $o(1)$.
Therefore, comparing RNN and CNN, the transformer can learn the long dependencies in the text more easily, and can fully implement parallel computing, accelerate the computational efficiency of the model, and thus obtain better results and the favor of algorithm personnel.

- Input

In order for the BERT model to be able to handle a variety of NLP tasks more flexibly, the model needs to be able to receive a single sentence or sentence pair as input. Just as the above-mentioned transformer receives the vector input of the word sequence $x=(x_1,…,x_n)$, how do you convert a single sentence or sentence pair into a vector in BERT?
As shown in the figure below, first get the word/word sequence of the sentence. And add a special [SEP][SEP] at the end of the sentence to mark the end of a sentence. Add [CLS][CLS] at the beginning of the sequence to learn the semantic representation of the entire sentence or sentence pair. More about the role of [CLS][CLS] will be introduced in the pre-training and fine-tunging sections below.

![image.png](./../imgs/bert_embedding.png)

• Token embedding


For the word/word sequence, the corresponding token embedding will be obtained by looking up the table. It is worth noting that the special [CLS][CLS] and [SEP][SEP] are also in the vocabulary, have their own embedding, and all token embedding can be learned.

• Positional embedding


Since BERT uses transformer as the encoder, in order to preserve the position information of the text, additional positional embedding is needed. The BERT model uses Learned Positional Embedding to encode absolute positions. Directly initialize a postion embedding randomly at different positions, add it to the token embedding and input the model as a parameter for training.
For more analysis of positional embedding, please refer to here.

• Segment embedding


Because the input of BERT can be in the form of sentence pairs, although [SEP][SEP] has been used to distinguish the end of the sentence on the input token, since the core mechanism attention number in the transformer will ignore the sequence relationship, we also need to add Segment embedding, used to distinguish the segment to which each token belongs. E_AEA identifies the token from sentence A, and E_BEB identifies the token from sentence B.
In general, BERT receives text input, through word segmentation, for each token, the sum of its corresponding token embedding, segment embedding, and position embedding is input into the model as the token's representation vector.


• Output

The transformer receives the vector representation of the word sequence with the length of $n$ $x = (x_1,...,x_n)$, and outputs the encoded vector $z = (z_1, …, z_n)$, each token $x_i$ is combined. The context is represented as a vector $z_i$. Each sub-words and special [CLS][CLS] vector representations will be used for training or prediction in different tasks.

$Author$: Nelson LIN

$Email$: nelsonlin0321@outlook.com